# SQLite Database

#### Inbuilt Database in Python Standard Library - SQLite3

- **Most widely deployed** database in the world
- Written in **C-language** as a library
- Provides a **SQL database engine**
- **Cross-platform** compatibility
- Databases are stored as **files on disk**
  
#### Key Features:
- **Lightweight**, serverless, self-contained, and zero-configuration
- **ACID compliant**, ensuring:
  - **Atomicity**: Transactions are all-or-nothing
  - **Consistency**: Database remains consistent before and after a transaction
  - **Isolation**: Transactions are independent of each other
  - **Durability**: Once a transaction is committed, it remains permanent even in case of a system failure


### Connection Object

- Provides the means to **interact with the database**.

### Cursor Object

- An **interface** provided by the database API.
- Acts as a **pointer** to a memory location where the result set is stored.

#### Key Methods:
- **`execute()`**: Executes SQL queries.
- **`fetchall()`**: Retrieves all results from the executed query.
- Can **iterate over results** for further processing.

In [ ]:
#@title { vertical-output: true}

from PIL import Image  # pip install Pillow
import datetime
import sqlite3
import shutil
import time
import math
import io
import os

# Connect to a database (creates if not exists)
conn = sqlite3.connect('example.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Execute a SQL command to create a table
def create_table():
    cursor.execute('''CREATE TABLE IF NOT EXISTS users
                    (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)''')

# Insert a single row into the table
def insert_user(name, age):
    cursor.execute("INSERT INTO users (name, age) VALUES (?, ?)", (name, age))

# Insert multiple rows into the table
def insert_many_users(users):
    cursor.executemany("INSERT INTO users (name, age) VALUES (?, ?)", users)

# Query all rows from the table
def get_all_users():
    cursor.execute("SELECT * FROM users")
    return cursor.fetchall()

# Query a single row by ID
def get_user_by_id(user_id):
    cursor.execute("SELECT * FROM users WHERE id=?", (user_id,))
    return cursor.fetchone()

# Update a user's information
def update_user(user_id, name, age):
    cursor.execute("UPDATE users SET name=?, age=? WHERE id=?", (name, age, user_id))

# Delete a user by ID
def delete_user(user_id):
    cursor.execute("DELETE FROM users WHERE id=?", (user_id,))

# Commit changes and close the connection
def close_connection():
    conn.commit()
    conn.close()

# Using transactions

# Start a transaction
def start_transaction():
    conn.execute("BEGIN TRANSACTION")

# Commit a transaction
def commit_transaction():
    conn.commit()

# Rollback a transaction
def rollback_transaction():
    conn.rollback()

# Example of using a transaction
def transaction_example():
    try:
        start_transaction()
        insert_user("Alice", 30)
        insert_user("Bob", 25)
        commit_transaction()
        print("Transaction committed successfully")
    except sqlite3.Error as e:
        rollback_transaction()
        print(f"Transaction rolled back. Error: {e}")

# Context manager for transactions
class SQLiteTransaction:
    def __init__(self, connection):
        self.connection = connection

    def __enter__(self):
        self.connection.execute("BEGIN TRANSACTION")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            self.connection.commit()
        else:
            self.connection.rollback()

# Example of using the transaction context manager
def transaction_context_example():
    with SQLiteTransaction(conn):
        insert_user("Charlie", 35)
        insert_user("David", 28)
    print("Transaction completed")



# Error handling

# Custom exception for database operations
class DatabaseError(Exception):
    pass

# Wrapper function for database operations with error handling
def db_operation(operation):
    def wrapper(*args, **kwargs):
        try:
            return operation(*args, **kwargs)
        except sqlite3.Error as e:
            raise DatabaseError(f"Database operation failed: {e}")
    return wrapper

# Example of using the wrapper for error handling
@db_operation
def insert_user_safe(name, age):
    cursor.execute("INSERT INTO users (name, age) VALUES (?, ?)", (name, age))

# Function to handle specific SQLite errors
def handle_sqlite_error(error):
    if isinstance(error, sqlite3.IntegrityError):
        print("Integrity Error: Possible duplicate entry or constraint violation")
    elif isinstance(error, sqlite3.OperationalError):
        print("Operational Error: Database might be locked or disk full")
    elif isinstance(error, sqlite3.ProgrammingError):
        print("Programming Error: SQL syntax error or missing schema")
    else:
        print(f"An unexpected error occurred: {error}")

# Example of using try-except for error handling
def query_with_error_handling(query, params=None):
    try:
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        return cursor.fetchall()
    except sqlite3.Error as e:
        handle_sqlite_error(e)
        return None

# Example usage of error handling
def error_handling_example():
    try:
        # Attempting to insert a duplicate primary key
        insert_user_safe("Eve", 40)
        insert_user_safe("Eve", 41)
    except DatabaseError as e:
        print(e)

    # Querying with potential errors
    result = query_with_error_handling("SELECT * FROM non_existent_table")
    if result is not None:
        print(result)




# Working with SQLite data types

# Create a table with various data types
def create_data_types_table():
    cursor.execute('''CREATE TABLE IF NOT EXISTS data_types_example
                    (id INTEGER PRIMARY KEY,
                     text_col TEXT,
                     int_col INTEGER,
                     real_col REAL,
                     blob_col BLOB,
                     null_col NULL,
                     date_col DATE,
                     datetime_col DATETIME)''')

# Insert data with various types
def insert_data_types_example():
    current_date = datetime.date.today()
    current_datetime = datetime.datetime.now()
    binary_data = b'Binary data example'

    cursor.execute('''INSERT INTO data_types_example
                    (text_col, int_col, real_col, blob_col, null_col, date_col, datetime_col)
                    VALUES (?, ?, ?, ?, ?, ?, ?)''',
                    ("Text example", 42, 3.14, binary_data, None, current_date, current_datetime))

# Query and display data types
def query_data_types():
    cursor.execute("SELECT * FROM data_types_example")
    row = cursor.fetchone()
    if row:
        print("Text:", row[1])
        print("Integer:", row[2])
        print("Real:", row[3])
        print("BLOB:", row[4])
        print("NULL:", row[5])
        print("Date:", row[6])
        print("DateTime:", row[7])

# Convert SQLite date string to Python date object
def sqlite_date_to_python(date_string):
    return datetime.datetime.strptime(date_string, "%Y-%m-%d").date()

# Convert SQLite datetime string to Python datetime object
def sqlite_datetime_to_python(datetime_string):
    return datetime.datetime.strptime(datetime_string, "%Y-%m-%d %H:%M:%S")

# Example of working with dates
def date_example():
    cursor.execute("SELECT date_col FROM data_types_example")
    date_string = cursor.fetchone()[0]
    python_date = sqlite_date_to_python(date_string)
    print(f"SQLite date: {date_string}, Python date: {python_date}")

# Example of working with datetimes
def datetime_example():
    cursor.execute("SELECT datetime_col FROM data_types_example")
    datetime_string = cursor.fetchone()[0]
    python_datetime = sqlite_datetime_to_python(datetime_string)
    print(f"SQLite datetime: {datetime_string}, Python datetime: {python_datetime}")


# Using parameterized queries

# Insert a user with parameterized query
def insert_user_parameterized(name, age):
    cursor.execute("INSERT INTO users (name, age) VALUES (?, ?)", (name, age))

# Query users with parameterized query
def get_users_by_age_range(min_age, max_age):
    cursor.execute("SELECT * FROM users WHERE age BETWEEN ? AND ?", (min_age, max_age))
    return cursor.fetchall()

# Update user with parameterized query
def update_user_parameterized(user_id, new_name, new_age):
    cursor.execute("UPDATE users SET name = ?, age = ? WHERE id = ?", (new_name, new_age, user_id))

# Delete user with parameterized query
def delete_user_parameterized(user_id):
    cursor.execute("DELETE FROM users WHERE id = ?", (user_id,))

# Example of using named parameters
def insert_user_named_params(name, age):
    cursor.execute("INSERT INTO users (name, age) VALUES (:name, :age)", {"name": name, "age": age})

# Query with named parameters
def get_users_by_name_and_age(name, min_age):
    cursor.execute("SELECT * FROM users WHERE name = :name AND age >= :min_age",
                   {"name": name, "min_age": min_age})
    return cursor.fetchall()

# Example of using parameterized queries with IN clause
def get_users_by_names(names):
    placeholders = ', '.join(['?'] * len(names))
    query = f"SELECT * FROM users WHERE name IN ({placeholders})"
    cursor.execute(query, names)
    return cursor.fetchall()

# Example of using parameterized queries with LIKE
def search_users_by_name_pattern(pattern):
    cursor.execute("SELECT * FROM users WHERE name LIKE ?", (f"%{pattern}%",))
    return cursor.fetchall()

# Example usage of parameterized queries
def parameterized_queries_example():
    insert_user_parameterized("Alice", 30)
    insert_user_named_params("Bob", 25)

    print("Users aged 20-35:", get_users_by_age_range(20, 35))
    print("Users named Bob aged 20+:", get_users_by_name_and_age("Bob", 20))
    print("Users named Alice or Bob:", get_users_by_names(["Alice", "Bob"]))
    print("Users with 'li' in their name:", search_users_by_name_pattern("li"))

    update_user_parameterized(1, "Alicia", 31)
    delete_user_parameterized(2)


# Executing complex queries (JOIN, GROUP BY, etc.)

# Create a new table for demonstrating complex queries
def create_orders_table():
    cursor.execute('''CREATE TABLE IF NOT EXISTS orders
                    (id INTEGER PRIMARY KEY,
                     user_id INTEGER,
                     product TEXT,
                     quantity INTEGER,
                     price REAL,
                     FOREIGN KEY (user_id) REFERENCES users(id))''')

# Insert sample data into the orders table
def insert_sample_orders():
    orders = [
        (1, "Widget A", 2, 10.99),
        (1, "Widget B", 1, 24.99),
        (2, "Widget A", 3, 10.99),
        (3, "Widget C", 2, 15.99),
        (2, "Widget B", 2, 24.99),
    ]
    cursor.executemany("INSERT INTO orders (user_id, product, quantity, price) VALUES (?, ?, ?, ?)", orders)

# Example of INNER JOIN query
def get_user_orders():
    query = '''
    SELECT users.name, orders.product, orders.quantity, orders.price
    FROM users
    INNER JOIN orders ON users.id = orders.user_id
    '''
    cursor.execute(query)
    return cursor.fetchall()

# Example of LEFT JOIN query
def get_all_users_with_orders():
    query = '''
    SELECT users.name, COALESCE(SUM(orders.quantity * orders.price), 0) as total_spent
    FROM users
    LEFT JOIN orders ON users.id = orders.user_id
    GROUP BY users.id
    '''
    cursor.execute(query)
    return cursor.fetchall()

# Example of GROUP BY and HAVING query
def get_high_value_customers(min_total):
    query = '''
    SELECT users.name, SUM(orders.quantity * orders.price) as total_spent
    FROM users
    INNER JOIN orders ON users.id = orders.user_id
    GROUP BY users.id
    HAVING total_spent >= ?
    ORDER BY total_spent DESC
    '''
    cursor.execute(query, (min_total,))
    return cursor.fetchall()

# Example of subquery
def get_users_with_above_average_orders():
    query = '''
    SELECT name, total_spent
    FROM (
        SELECT users.name, SUM(orders.quantity * orders.price) as total_spent
        FROM users
        INNER JOIN orders ON users.id = orders.user_id
        GROUP BY users.id
    )
    WHERE total_spent > (
        SELECT AVG(total_spent)
        FROM (
            SELECT users.id, SUM(orders.quantity * orders.price) as total_spent
            FROM users
            INNER JOIN orders ON users.id = orders.user_id
            GROUP BY users.id
        )
    )
    '''
    cursor.execute(query)
    return cursor.fetchall()

# Example usage of complex queries
def complex_queries_example():
    create_orders_table()
    insert_sample_orders()

    print("User orders:")
    for row in get_user_orders():
        print(f"{row[0]} ordered {row[1]} (Quantity: {row[2]}, Price: ${row[3]:.2f})")

    print("\nAll users with total spent:")
    for row in get_all_users_with_orders():
        print(f"{row[0]} - Total spent: ${row[1]:.2f}")

    print("\nHigh value customers (min total $50):")
    for row in get_high_value_customers(50):
        print(f"{row[0]} - Total spent: ${row[1]:.2f}")

    print("\nUsers with above average order totals:")
    for row in get_users_with_above_average_orders():
        print(f"{row[0]} - Total spent: ${row[1]:.2f}")



# Using SQLite functions

# Example of using built-in SQLite functions
def use_builtin_functions():
    query = '''
    SELECT
        name,
        UPPER(name) as uppercase_name,
        LENGTH(name) as name_length,
        ROUND(AVG(price), 2) as avg_price,
        COUNT(*) as order_count,
        MIN(price) as min_price,
        MAX(price) as max_price,
        SUM(quantity) as total_quantity
    FROM users
    JOIN orders ON users.id = orders.user_id
    GROUP BY users.id
    '''
    cursor.execute(query)
    return cursor.fetchall()

# Example of using date and time functions
def use_date_time_functions():
    query = '''
    SELECT
        DATE('now') as current_date,
        TIME('now') as current_time,
        DATETIME('now') as current_datetime,
        DATE('now', '+1 day') as tomorrow,
        DATE('now', 'start of month', '+1 month', '-1 day') as last_day_of_month
    '''
    cursor.execute(query)
    return cursor.fetchone()

# Create a custom SQLite function
def create_custom_function():
    def calculate_discount(price, discount_percent):
        return price * (1 - discount_percent / 100)

    conn.create_function("calculate_discount", 2, calculate_discount)

# Use the custom function in a query
def use_custom_function():
    create_custom_function()
    query = '''
    SELECT
        product,
        price,
        calculate_discount(price, 10) as discounted_price
    FROM orders
    '''
    cursor.execute(query)
    return cursor.fetchall()

# Create a custom aggregate function
def create_custom_aggregate():
    class Varianceparameter:
        def __init__(self):
            self.count = 0
            self.sum = 0
            self.sum_sq = 0

        def step(self, value):
            self.count += 1
            self.sum += value
            self.sum_sq += value * value

        def finalize(self):
            if self.count < 2:
                return None
            mean = self.sum / self.count
            variance = (self.sum_sq / self.count) - (mean * mean)
            return variance

    conn.create_aggregate("VARIANCE", 1, Varianceparameter)

# Use the custom aggregate function in a query
def use_custom_aggregate():
    create_custom_aggregate()
    query = '''
    SELECT
        AVG(price) as avg_price,
        VARIANCE(price) as price_variance
    FROM orders
    '''
    cursor.execute(query)
    return cursor.fetchone()

# Example usage of SQLite functions
def sqlite_functions_example():
    print("Built-in function examples:")
    for row in use_builtin_functions():
        print(row)

    print("\nDate and time function examples:")
    print(use_date_time_functions())

    print("\nCustom discount function example:")
    for row in use_custom_function():
        print(f"Product: {row[0]}, Original price: ${row[1]:.2f}, Discounted price: ${row[2]:.2f}")

    print("\nCustom aggregate function example:")
    result = use_custom_aggregate()
    print(f"Average price: ${result[0]:.2f}, Price variance: {result[1]:.2f}")



# Working with BLOBs (Binary Large Objects)

# Create a table to store images
def create_images_table():
    cursor.execute('''CREATE TABLE IF NOT EXISTS images
                    (id INTEGER PRIMARY KEY,
                     name TEXT,
                     data BLOB)''')

# Function to convert an image to binary data
def get_image_data(file_path):
    with open(file_path, 'rb') as file:
        blob_data = file.read()
    return blob_data

# Insert an image into the database
def insert_image(name, file_path):
    blob_data = get_image_data(file_path)
    cursor.execute("INSERT INTO images (name, data) VALUES (?, ?)", (name, blob_data))
    conn.commit()

# Retrieve an image from the database
def get_image(image_id):
    cursor.execute("SELECT name, data FROM images WHERE id = ?", (image_id,))
    name, blob_data = cursor.fetchone()
    return name, blob_data

# Save a retrieved image to a file
def save_image(name, blob_data, output_path):
    with open(output_path, 'wb') as file:
        file.write(blob_data)

# Function to resize an image before storing (to reduce BLOB size)
def resize_image(file_path, max_size=(800, 600)):
    with Image.open(file_path) as img:
        img.thumbnail(max_size)
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='PNG')
        return img_byte_arr.getvalue()

# Insert a resized image into the database
def insert_resized_image(name, file_path, max_size=(800, 600)):
    blob_data = resize_image(file_path, max_size)
    cursor.execute("INSERT INTO images (name, data) VALUES (?, ?)", (name, blob_data))
    conn.commit()

# Function to display basic image information
def get_image_info(image_id):
    cursor.execute("SELECT name, length(data) as size FROM images WHERE id = ?", (image_id,))
    name, size = cursor.fetchone()
    return f"Image: {name}, Size: {size} bytes"

# Example usage of BLOB operations
def blob_operations_example():
    create_images_table()

    # Insert an image
    insert_image("example.jpg", "path/to/example.jpg")
    print("Image inserted successfully.")

    # Retrieve and save an image
    name, blob_data = get_image(1)
    save_image(name, blob_data, "path/to/output/retrieved_image.jpg")
    print(f"Image '{name}' retrieved and saved.")

    # Insert a resized image
    insert_resized_image("resized_example.jpg", "path/to/example.jpg")
    print("Resized image inserted successfully.")

    # Display image information
    print(get_image_info(1))
    print(get_image_info(2))

# Function to search for images by name
def search_images(name_pattern):
    cursor.execute("SELECT id, name FROM images WHERE name LIKE ?", (f"%{name_pattern}%",))
    return cursor.fetchall()

# Function to delete an image
def delete_image(image_id):
    cursor.execute("DELETE FROM images WHERE id = ?", (image_id,))
    conn.commit()

# Creating and using indexes

# Create an index on the 'name' column of the 'users' table
def create_name_index():
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_users_name ON users(name)")

# Create a composite index on 'user_id' and 'product' columns of the 'orders' table
def create_order_composite_index():
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_orders_user_product ON orders(user_id, product)")

# Create a unique index on the 'email' column of the 'users' table
def create_unique_email_index():
    cursor.execute("CREATE UNIQUE INDEX IF NOT EXISTS idx_users_email ON users(email)")

# Drop an index
def drop_index(index_name):
    cursor.execute(f"DROP INDEX IF EXISTS {index_name}")

# List all indexes in the database
def list_indexes():
    cursor.execute("SELECT name FROM sqlite_master WHERE type='index'")
    return [row[0] for row in cursor.fetchall()]

# Function to measure query execution time
def measure_query_time(query, params=None):
    start_time = time.time()
    if params:
        cursor.execute(query, params)
    else:
        cursor.execute(query)
    cursor.fetchall()
    end_time = time.time()
    return end_time - start_time

# Compare query performance with and without an index
def compare_index_performance():
    # Ensure we have some data
    cursor.executemany("INSERT INTO users (name, age) VALUES (?, ?)",
                       [("User" + str(i), i) for i in range(1000)])

    # Query without index
    query = "SELECT * FROM users WHERE name = ?"
    no_index_time = measure_query_time(query, ("User500",))

    # Create index
    create_name_index()

    # Query with index
    with_index_time = measure_query_time(query, ("User500",))

    print(f"Query time without index: {no_index_time:.6f} seconds")
    print(f"Query time with index: {with_index_time:.6f} seconds")
    print(f"Performance improvement: {(1 - with_index_time/no_index_time) * 100:.2f}%")

# Example of using EXPLAIN QUERY PLAN
def explain_query_plan(query, params=None):
    if params:
        cursor.execute(f"EXPLAIN QUERY PLAN {query}", params)
    else:
        cursor.execute(f"EXPLAIN QUERY PLAN {query}")
    return cursor.fetchall()

# Example usage of index operations
def index_operations_example():
    # Create indexes
    create_name_index()
    create_order_composite_index()
    create_unique_email_index()

    print("Created indexes.")

    # List indexes
    print("Indexes in the database:")
    for index in list_indexes():
        print(f"- {index}")

    # Compare performance
    compare_index_performance()

    # Explain query plan
    query = "SELECT * FROM users WHERE name = ?"
    plan = explain_query_plan(query, ("John Doe",))
    print("\nQuery plan:")
    for step in plan:
        print(f"- {step[3]}")

    # Drop an index
    drop_index("idx_users_name")
    print("\nDropped idx_users_name index.")

    print("\nRemaining indexes:")
    for index in list_indexes():
        print(f"- {index}")


# Backing up and restoring databases

# Backup the entire database
def backup_database(source_db, backup_dir):
    if not os.path.exists(backup_dir):
        os.makedirs(backup_dir)

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_file = os.path.join(backup_dir, f"backup_{timestamp}.db")

    # Create a new connection to the source database
    with sqlite3.connect(source_db) as source:
        # Open the backup file
        with sqlite3.connect(backup_file) as target:
            source.backup(target)

    print(f"Database backed up to: {backup_file}")
    return backup_file

# Restore database from a backup
def restore_database(backup_file, restore_path):
    shutil.copy(backup_file, restore_path)
    print(f"Database restored to: {restore_path}")

# Backup specific tables
def backup_tables(source_db, backup_dir, tables):
    if not os.path.exists(backup_dir):
        os.makedirs(backup_dir)

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_file = os.path.join(backup_dir, f"backup_tables_{timestamp}.db")

    with sqlite3.connect(source_db) as source, sqlite3.connect(backup_file) as target:
        for table in tables:
            # Copy table structure
            source.execute(f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{table}'")
            create_table_sql = source.fetchone()[0]
            target.execute(create_table_sql)

            # Copy table data
            source.execute(f"SELECT * FROM {table}")
            rows = source.fetchall()
            target.executemany(f"INSERT INTO {table} VALUES ({','.join(['?' for _ in rows[0]])})", rows)

        target.commit()

    print(f"Tables {', '.join(tables)} backed up to: {backup_file}")
    return backup_file

# Export database to SQL script
def export_to_sql(source_db, output_file):
    with sqlite3.connect(source_db) as conn:
        with open(output_file, 'w') as f:
            for line in conn.iterdump():
                f.write(f"{line}\n")
    print(f"Database exported to SQL script: {output_file}")

# Import database from SQL script
def import_from_sql(sql_file, target_db):
    with sqlite3.connect(target_db) as conn:
        with open(sql_file, 'r') as f:
            sql_script = f.read()
        conn.executescript(sql_script)
    print(f"Database imported from SQL script to: {target_db}")

# Example usage of backup and restore operations
def backup_restore_example():
    # Backup the entire database
    backup_file = backup_database("example.db", "backups")

    # Restore the database
    restore_database(backup_file, "restored_example.db")

    # Backup specific tables
    backup_tables("example.db", "backups", ["users", "orders"])

    # Export to SQL script
    export_to_sql("example.db", "example_dump.sql")

    # Import from SQL script
    import_from_sql("example_dump.sql", "imported_example.db")

# Function to verify database integrity
def check_database_integrity(db_file):
    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()
        cursor.execute("PRAGMA integrity_check")
        result = cursor.fetchone()[0]
        if result == "ok":
            print(f"Database integrity check passed for {db_file}")
        else:
            print(f"Database integrity check failed for {db_file}. Result: {result}")


# Main function to demonstrate all SQLite3 operations

def main():
    # Set up the database
    conn = sqlite3.connect('test_database.db')
    cursor = conn.cursor()

    try:
        # Basic CRUD operations
        create_table()
        insert_user("Alice", 30)
        insert_user("Bob", 25)
        insert_user("Charlie", 35)
        print("All users:")
        for user in get_all_users():
            print(user)

        print("\nUser with id 2:")
        print(get_user_by_id(2))

        update_user(2, "Bobby", 26)
        print("\nUpdated user with id 2:")
        print(get_user_by_id(2))

        delete_user(3)
        print("\nAll users after deletion:")
        for user in get_all_users():
            print(user)

        # Transactions
        print("\nTesting transactions:")
        transaction_example()

        # Error handling
        print("\nTesting error handling:")
        try:
            insert_user_safe("Bobby", 26)  # Should raise an error due to duplicate name
        except DatabaseError as e:
            print(f"Caught expected error: {e}")

        # Complex queries
        create_orders_table()
        insert_sample_orders()
        print("\nTesting complex queries:")
        complex_queries_example()

        # SQLite functions
        print("\nTesting SQLite functions:")
        sqlite_functions_example()

        # BLOBs
        print("\nTesting BLOB operations:")
        blob_operations_example()

        # Indexes
        print("\nTesting index operations:")
        index_operations_example()

        # Backup and restore
        print("\nTesting backup and restore operations:")
        backup_restore_example()

        # Check database integrity
        check_database_integrity('test_database.db')

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()
        print("\nDatabase connection closed.")

    # Clean up test files
    cleanup_files = ['test_database.db', 'restored_example.db', 'imported_example.db', 'example_dump.sql']
    for file in cleanup_files:
        if os.path.exists(file):
            os.remove(file)
    print("Test files cleaned up.")

if __name__ == "__main__":
    main()

All users:
(1, 'Alice', 30)
(2, 'Bob', 25)
(3, 'Charlie', 35)

User with id 2:
(2, 'Bob', 25)

Updated user with id 2:
(2, 'Bobby', 26)

All users after deletion:
(1, 'Alice', 30)
(2, 'Bobby', 26)

Testing transactions:
Transaction rolled back. Error: cannot start a transaction within a transaction

Testing error handling:

Testing complex queries:
User orders:
Bobby ordered Widget A (Quantity: 2, Price: $10.99)
Bobby ordered Widget B (Quantity: 1, Price: $24.99)
Bobby ordered Widget A (Quantity: 2, Price: $10.99)
Bobby ordered Widget B (Quantity: 1, Price: $24.99)

All users with total spent:
Bobby - Total spent: $93.94

High value customers (min total $50):
Bobby - Total spent: $93.94

Users with above average order totals:

Testing SQLite functions:
Built-in function examples:
('Bobby', 'BOBBY', 5, 17.99, 4, 10.99, 24.99, 6)

Date and time function examples:
('2024-09-15', '05:43:50', '2024-09-15 05:43:50', '2024-09-16', '2024-09-30')

Custom discount function example:
Product: Widg

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/example.jpg'

### Basic Database Operations with SQLite

Create a new SQLite database named `school.db`.

#### Tables

1. **Students**
   - `id`: INTEGER, Primary Key
   - `name`: TEXT
   - `age`: INTEGER
   - `grade`: TEXT

2. **Courses**
   - `course_id`: INTEGER, Primary Key
   - `course_name`: TEXT
   - `instructor`: TEXT

#### Insert Data

- Insert 5 student records into the `Students` table
- Insert 3 course records into the `Courses` table

#### Query Database

- Retrieve all students who are in grade 'A'
- List all courses along with their instructors, in ascending order of course names

#### Update Data

- Change the grade of a particular student to 'A'
- Update the instructor for a course with `course_id = 2`

#### Delete a Record

- Delete a student from the `Students` table where the student's `id` is 4

#### Drop Tables

- Drop both tables after completing the tasks


#### Exception handling

- Try
- Except
- Finally

In [ ]:
#@title { vertical-output: true}

import sqlite3
import random
import string

try:
    conn = sqlite3.connect('school1.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Students (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            age INTEGER,
            grade TEXT,
            course_id INTEGER,
            FOREIGN KEY (course_id) REFERENCES Courses(course_id)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Courses (
            course_id INTEGER PRIMARY KEY AUTOINCREMENT,
            course_name TEXT,
            instructor TEXT
        )
    ''')

    total_students = 5
    total_cources = 3
    for i in range(total_students):
        letters = string.ascii_letters
        name = ''.join(random.choice(letters) for _ in range(10))
        age = random.randint(10, 16)
        grade = random.choice(['A', 'B', 'C', 'D', 'E'])
        course_id = random.randint(1, total_cources)
        cursor.execute("INSERT INTO Students (name, age, grade, course_id) VALUES (?, ?, ?, ?)", (name, age, grade, course_id))

    for i in range(total_cources):
        letters = string.ascii_letters
        course_name = ''.join(random.choice(letters) for _ in range(5))
        instructor = ''.join(random.choice(letters) for _ in range(10))
        cursor.execute("INSERT INTO Courses (course_name, instructor) VALUES (?, ?)", (course_name, instructor))

    print("\nAll the students")
    students = cursor.execute("SELECT * FROM Students")
    for std in students:
        print(std)

    print("\nAll the students woth A grade")
    toppers = cursor.execute("SELECT * FROM Students WHERE grade=?", ('A',))
    for std in toppers:
        print(std)

    print("\nAll the courses ascending")
    courses = cursor.execute("SELECT * FROM Courses ORDER BY course_name ASC")
    for course in courses:
        print(course)

    cursor.execute("UPDATE Students SET grade='A' WHERE id=?", (1,))
    cursor.execute("UPDATE Courses SET instructor=? WHERE course_id=?", ('New Instructor', 2))
    cursor.execute("DELETE FROM Students WHERE id=?", (4,))

    print("\nAll the tables")
    res = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    for r in res:
        print(r)


    print("Dropping all the tables")
    cursor.execute("DROP TABLE IF EXISTS Students")
    cursor.execute("DROP TABLE IF EXISTS Courses")

    conn.commit()

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if conn:
        conn.close()


All the students
(1, 'AzNcNHgaJT', 10, 'B', 1)
(2, 'qRQidcNfyv', 13, 'A', 2)
(3, 'yMyNcqZouz', 14, 'E', 3)
(4, 'bvOfeNGPkP', 11, 'E', 1)
(5, 'YqVQcilffa', 15, 'E', 1)

All the students woth A grade
(2, 'qRQidcNfyv', 13, 'A', 2)

All the courses ascending
(2, 'KGkwi', 'lTvhmKWgQJ')
(3, 'VEFZK', 'YfOrExDbCw')
(1, 'XaAtW', 'OlDhGasLnX')

All the tables
('sqlite_sequence',)
('Students',)
('Courses',)
Dropping all the tables


In [ ]:
#@title { vertical-output: true}

import sqlite3
import random
import string

try:
    conn = sqlite3.connect('school1.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Courses (
            course_id INTEGER PRIMARY KEY AUTOINCREMENT,
            course_name TEXT
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Students (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            age INTEGER,
            marks INTEGER,
            course_id INTEGER,
            FOREIGN KEY (course_id) REFERENCES Courses(course_id)
        )
    ''')

    total_students = 5
    total_courses = 3

    # Insert courses first
    for i in range(total_courses):
        letters = string.ascii_letters
        course_name = ''.join(random.choice(letters) for _ in range(5))
        cursor.execute("INSERT INTO Courses (course_name) VALUES (?)", (course_name,))
        conn.commit()

    # Then insert students
    for i in range(total_students):
        letters = string.ascii_letters
        name = ''.join(random.choice(letters) for _ in range(10))
        age = random.randint(10, 16)
        marks = random.randint(0, 100)

        # Ensure course_id exists in Courses table
        course_id = random.randint(1, total_courses)
        cursor.execute("INSERT INTO Students (name, age, marks, course_id) VALUES (?, ?, ?, ?)", (name, age, marks, course_id))
        conn.commit()

    print("\nAll the students")
    students = cursor.execute("SELECT * FROM Students")
    for std in students:
        print(std)

    print("\nAll the courses ascending")
    courses = cursor.execute("SELECT * FROM Courses ORDER BY course_name ASC")
    for course in courses:
        print(course)

    print("\nAll the students")
    students = cursor.execute('''
        SELECT s.id, s.name,s.marks, c.course_name
        FROM Students as s
        JOIN Courses as c
        ON s.course_id = c.course_id
    ''')
    for std in students:
        print(std)

    print("Dropping all the tables")
    cursor.execute("DROP TABLE IF EXISTS Students")
    cursor.execute("DROP TABLE IF EXISTS Courses")
    conn.commit()

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if conn:
        conn.close()


All the students
(1, 'wxSQnTwZUq', 11, 36, 2)
(2, 'cyUMQfXWad', 11, 45, 3)
(3, 'yaiHpPlFDW', 10, 47, 2)
(4, 'HmzmGWgcGQ', 12, 43, 1)
(5, 'mTPVWrsgfL', 12, 48, 2)

All the courses ascending
(2, 'eMzir')
(3, 'nuEPQ')
(1, 'orKNF')

All the students
(1, 'wxSQnTwZUq', 36, 'eMzir')
(2, 'cyUMQfXWad', 45, 'nuEPQ')
(3, 'yaiHpPlFDW', 47, 'eMzir')
(4, 'HmzmGWgcGQ', 43, 'orKNF')
(5, 'mTPVWrsgfL', 48, 'eMzir')
Dropping all the tables


In [ ]:
#@title { vertical-output: true}
import sqlite3

try:
    # 1. Establish a connection to the database
    conn = sqlite3.connect('company.db')
    conn.execute("PRAGMA foreign_keys = 1")  # Enable foreign key support
    cursor = conn.cursor()

    # 2. Create 'departments' table with primary key
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS departments (
            department_id INTEGER PRIMARY KEY,
            department_name TEXT NOT NULL
        )
    ''')

    # 3. Create 'employees' table with primary key and foreign key
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS employees (
            employee_id INTEGER PRIMARY KEY,
            employee_name TEXT NOT NULL,
            department_id INTEGER,
            FOREIGN KEY (department_id) REFERENCES departments (department_id)
        )
    ''')

    # 4. Create indexes
    cursor.execute('''
        CREATE INDEX IF NOT EXISTS idx_employee_name
        ON employees (employee_name)
    ''')

    cursor.execute('''
        CREATE INDEX IF NOT EXISTS idx_department_name
        ON departments (department_name)
    ''')

    # 5. Insert data into 'departments'
    cursor.execute("INSERT INTO departments (department_name) VALUES ('Human Resources')")
    cursor.execute("INSERT INTO departments (department_name) VALUES ('Engineering')")
    cursor.execute("INSERT INTO departments (department_name) VALUES ('Sales')")

    # 6. Insert data into 'employees'
    cursor.execute("INSERT INTO employees (employee_name, department_id) VALUES ('Mayura', 1)")
    cursor.execute("INSERT INTO employees (employee_name, department_id) VALUES ('Mayuresh', 2)")
    cursor.execute("INSERT INTO employees (employee_name, department_id) VALUES ('Sugandha', 2)")
    cursor.execute("INSERT INTO employees (employee_name, department_id) VALUES ('Srujan', 1)")

    # 7. Commit the transaction to save changes
    conn.commit()

    print("Employees and their departments:")
    cursor.execute('''
        SELECT employee_id, employee_name, department_id
        FROM employees
    ''')
    results = cursor.fetchall()
    for row in results:
        print(row)

    # 8. Query with JOIN to demonstrate foreign key relationship
    print("\nEmployees and their departments (with JOIN):")
    cursor.execute('''
        SELECT employees.employee_name, departments.department_name
        FROM employees
        JOIN departments ON employees.department_id = departments.department_id
    ''')
    results = cursor.fetchall()
    for row in results:
        print(row)

except sqlite3.Error as error:
    print("Error while working with SQLite", error)
finally:
    print("\nAll the tables")
    res = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    for r in res:
        print(r)
    cursor.close()

Employees and their departments:
(1, 'Mayura', 1)
(2, 'Mayuresh', 2)
(3, 'Sugandha', 3)
(4, 'Srujan', 1)
(5, 'Mayura', 1)
(6, 'Mayuresh', 2)
(7, 'Sugandha', 2)
(8, 'Srujan', 1)

Employees and their departments (with JOIN):
('Mayura', 'Human Resources')
('Mayuresh', 'Engineering')
('Sugandha', 'Sales')
('Srujan', 'Human Resources')
('Mayura', 'Human Resources')
('Mayuresh', 'Engineering')
('Sugandha', 'Engineering')
('Srujan', 'Human Resources')

All the tables
('departments',)
('employees',)


# ORM - SQLAlchemy

In [ ]:
#@title { vertical-output: true}

from sqlalchemy import create_engine, MetaData, Column, Integer, String, Float, Sequence
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import inspect

# Set up the database and the Base class
engine = create_engine('sqlite:///company_orm1.db')
Base = declarative_base()

# Define the Employee model
class Employee(Base):
    __tablename__ = 'employees'
    id = Column(Integer, Sequence('employee_id_seq'), primary_key=True)
    name = Column(String(50))
    age = Column(Integer)
    department = Column(String(50))
    salary = Column(Float)


# Create the tables
metadata = MetaData()
metadata.reflect(bind=engine)
# metadata.create_all(engine)
Base.metadata.create_all(engine)


# Accessing tables
employees = metadata.tables['employees']
# Reflect the employees table to retrieve its details
inspector = inspect(engine)
columns = inspector.get_columns('employees')

# Print table details
print("Table 'employees' Details:")
for column in columns:
    print(f"Column: {column['name']} Type: {column['type']}")

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Add employee records
employees = [
    Employee(name='Arun', age=30, department='HR', salary=70000),
    Employee(name='Radhika', age=25, department='IT', salary=80000),
    Employee(name='Chetan', age=35, department='Finance', salary=120800)
]

# Add all employee records to the session
session.add_all(employees)

# Commit the changes
session.commit()

# Query the database
all_employees = session.query(Employee).all()

# Print all employee records
for emp in all_employees:
    print(f"{emp.id}: {emp.name} - {emp.department} - ${emp.salary}")

Table 'employees' Details:
Column: id Type: INTEGER
Column: name Type: VARCHAR(50)
Column: age Type: INTEGER
Column: department Type: VARCHAR(50)
Column: salary Type: FLOAT
1: Arun - HR - $70000.0
2: Radhika - IT - $80000.0
3: Chetan - Finance - $120800.0
4: Arun - HR - $70000.0
5: Radhika - IT - $80000.0
6: Chetan - Finance - $120800.0


<ipython-input-76-cd3f0e2ad587>:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
#@title { vertical-output: true}
from sqlalchemy import create_engine, MetaData, Column, Integer, String, Sequence, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy import inspect
import string
import random

# Set up the database and the Base class
engine = create_engine('sqlite:///company_orm1.db')
Base = declarative_base()

# Define the Student model
class Students(Base):
    __tablename__ = 'students'
    id = Column(Integer, Sequence('student_id_seq'), primary_key=True)
    name = Column(String(50))
    marks = Column(Integer)
    course_id = Column(Integer, ForeignKey('course.id'))

    # Relationship to link students to courses
    course = relationship("Course", back_populates="students")

# Define the Course model
class Course(Base):
    __tablename__ = 'course'
    id = Column(Integer, Sequence('course_id_seq'), primary_key=True)
    course_name = Column(String(50))

    # Relationship to link courses to students
    students = relationship("Students", back_populates="course")

# Drop all existing tables (optional if needed)
Base.metadata.drop_all(engine)

# Create the tables
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Add course records
courses = []
total_courses = 3

for i in range(total_courses):
    letters = string.ascii_letters
    course_name = ''.join(random.choice(letters) for _ in range(5))
    course = Course(course_name=course_name)
    courses.append(course)

# Add all courses to the session
session.add_all(courses)

# Commit the courses to the database so we can retrieve their IDs
session.commit()

# Add student records
# Add student records
students = list([])
total_students = 5

for i in range(total_students):
    letters = string.ascii_letters
    name = ''.join(random.choice(letters) for _ in range(10))
    course_id = random.choice([course.id for course in courses])
    marks = random.randint(0, 100)
    student = Students(name=name, course_id=course_id, marks=marks)
    students.append(student)

# Add all students to the session
session.add_all(students)

# Commit the students to the database
session.commit()

# Printing all students and course
print("\nAll the students")
students = session.query(Students).all()
for std in students:
    print(f"{std.id}: {std.name} - {std.course.course_name} - {std.marks}")

print("\nAll the courses")
courses = session.query(Course).all()
for course in courses:
    print(f"{course.id}: {course.course_name}")

# Perform a join between Students and Course to retrieve student details with course names
results = session.query(Students, Course).join(Course, Students.course_id == Course.id).all()

# Print the results
print("\nStudent Details with Course Names (using Join):")
for student, course in results:
    print(f"Student ID: {student.id}, Name: {student.name}, Marks: {student.marks}, Course Name: {course.course_name}")


All the students
1: JgMmpkoJkZ - SNNpv - 30
2: DoCmlEbbTM - SNNpv - 55
3: uFdBmZtFMP - SNNpv - 20
4: WpDnZtTysv - ruLdt - 84
5: SHbIcPCMdE - WDthW - 23

All the courses
1: ruLdt
2: SNNpv
3: WDthW

Student Details with Course Names (using Join):
Student ID: 1, Name: JgMmpkoJkZ, Marks: 30, Course Name: SNNpv
Student ID: 2, Name: DoCmlEbbTM, Marks: 55, Course Name: SNNpv
Student ID: 3, Name: uFdBmZtFMP, Marks: 20, Course Name: SNNpv
Student ID: 4, Name: WpDnZtTysv, Marks: 84, Course Name: ruLdt
Student ID: 5, Name: SHbIcPCMdE, Marks: 23, Course Name: WDthW


<ipython-input-20-c656317bdeeb>:11: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Home Assignment

#### Retrieve Subject name using SQLAlchemy and Pandas

* Student_Details:

    * Stud_id: Integer
    * Student_Name: String
    * Subject_Name: String
    * Marks: Integer

* Insert records into the table

* Retrieve all details, check if insert is successful.

* Update marks for student with id = 2.

* Retrieve all details, check if Update query is successful.


In [14]:
#@title { vertical-output: true}
from sqlalchemy import create_engine, Column, Integer, String, Float, Sequence, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
import string
import random

engine = create_engine('sqlite:///studnets.db', echo=False)
Base = declarative_base()

class Student_Details(Base):
    __tablename__ = 'student_details'
    stud_id = Column(Integer, Sequence('student_id_seq'),  primary_key=True)
    subj_id = Column(Integer, ForeignKey('subject_details.subj_id'))
    stud_name = Column(String, nullable=False)
    marks = Column(Integer, nullable=False)
    subject = relationship("Subject_Details", back_populates="students")

class Subject_Details(Base):
    __tablename__ = 'subject_details'
    subj_id = Column(Integer, primary_key=True)
    subj_name = Column(String, nullable=False)
    students = relationship("Student_Details", back_populates="subject")

def create_table():
    Base.metadata.create_all(engine)
    tables = Base.metadata.tables.keys()
    print("Table Created.")
    for table in list(tables):
        print(table)


def insert_sample_data(number_of_students, number_of_sujects):
    Session = sessionmaker(bind=engine)
    session = Session()
    existing_data = session.query(Student_Details).first()
    if existing_data is None:
        students = []
        for i in range(number_of_students):
            letters = string.ascii_letters
            s_name = ''.join(random.choice(letters) for _ in range(10))
            s_subj_id = random.choice([x for x in range(number_of_sujects)])
            s_marks = random.randint(36, 100)
            student = Student_Details(stud_name=s_name, subj_id=s_subj_id, marks=s_marks)
            students.append(student)
        session.add_all(students)
        session.commit()
        print("Student sample data inserted successfully.")
    else:
        print("Student sample data already exists. Skipping insertion.")

    existing_data = session.query(Subject_Details).first()
    if existing_data is None:
        subjects = []
        for i in range(number_of_sujects):
            letters = string.ascii_letters
            s_name = ''.join(random.choice(letters) for _ in range(10))
            subject = Subject_Details(subj_name=s_name)
            subjects.append(subject)
        session.add_all(subjects)
        session.commit()
        print("Subject sample data inserted successfully.")
    else:
        print("Subject Sample data already exists. Skipping insertion.")

    session.close()

def read_data(table_name):
    df = pd.read_sql(table_name, con=engine)
    return df

def update_marks(df, stud_id, new_marks):
    df.loc[df['stud_id'] == stud_id, 'marks'] = new_marks
    return df


def write_data(df):
    df.to_sql('student_details', con=engine, if_exists='replace', index=False)
    print("Updated data written back to the database.")


def print_table_data(table_name):
    df = read_data(table_name)
    print(df)

def print_combine_table():
    df1 = read_data('student_details')
    df2 = read_data('subject_details')
    df = pd.merge(df1, df2, on='subj_id')
    print(df)


def delete_table():
    Base.metadata.drop_all(engine)
    tables = Base.metadata.tables.keys()
    print("Table Deleted.")
    for table in list(tables):
        print(table)

def delete_database():
    engine.dispose()
    print("Database deleted.")

def main():
    create_table()
    insert_sample_data(5,3)

    df = read_data('student_details')
    print("Original DataFrame:")
    print(df)

    df = update_marks(df, stud_id=2, new_marks=50)

    print("\nUpdated DataFrame:")
    print(df)

    write_data(df)
    print("\nData updated in the database.")

    print("\nUpdated DataFrame:")
    print_table_data('student_details')
    print_combine_table()
    print_table_data('subject_details')

    df = read_data('student_details')
    print("Updated DataFrame:")
    print(df)

    delete_table()
    delete_database()


if __name__ == '__main__':
    main()

Table Created.
student_details
subject_details
Student sample data inserted successfully.
Subject sample data inserted successfully.
Original DataFrame:
   stud_id  subj_id   stud_name  marks
0        1        1  tkchkxSqtc     91
1        2        2  STYRpkURQT     48
2        3        0  WlTQXBKnXc     55
3        4        2  mpOSePfagN     39
4        5        0  UjfIOeYmqu     93

Updated DataFrame:
   stud_id  subj_id   stud_name  marks
0        1        1  tkchkxSqtc     91
1        2        2  STYRpkURQT     50
2        3        0  WlTQXBKnXc     55
3        4        2  mpOSePfagN     39
4        5        0  UjfIOeYmqu     93
Updated data written back to the database.

Data updated in the database.

Updated DataFrame:
   stud_id  subj_id   stud_name  marks
0        1        1  tkchkxSqtc     91
1        2        2  STYRpkURQT     50
2        3        0  WlTQXBKnXc     55
3        4        2  mpOSePfagN     39
4        5        0  UjfIOeYmqu     93
   stud_id  subj_id   stud_name

<ipython-input-14-221681010cdf>:10: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Home Assignment

#### Retrieve Subject name using JOIN

* Subjects:
    * Subject_id: Integer (Primary Key)
    * Subject_Name: String
* Student_Details:
    * Stud_Id: Integer (Foreign Key References Students.Student_Name)
    * Subject_Id: Integer (Foreign Key References Subjects.Subject_id)
    * Marks: Integer
* Students:
    * Student id: Integer (Primary Key)
    * Student Name: String
* Insert records into all three tables
* Retrieve Student Name, Subject Name, Marks. Use JOIN.
* (Hint: Query will contain 2 joins)

In [20]:
#@title { vertical-output: true}

from sqlalchemy import create_engine, Column, Integer, String, Float, Sequence, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import pandas as pd
import random

engine = create_engine('sqlite:///students.db', echo=False)
Base = declarative_base()

class Student(Base):
    __tablename__ = 'student'
    student_id = Column(Integer, Sequence('student_id_seq'), primary_key=True)
    student_name = Column(String, nullable=False)
    student_details = relationship("StudentDetail", back_populates="student")

class Subject(Base):
    __tablename__ = 'subject'
    subject_id = Column(Integer, primary_key=True)
    subject_name = Column(String, nullable=False)
    student_details = relationship("StudentDetail", back_populates="subject")

class StudentDetail(Base):
    __tablename__ = 'student_detail'
    id = Column(Integer, Sequence('student_detail_id_seq'), primary_key=True)
    student_id = Column(Integer, ForeignKey('student.student_id'))
    subject_id = Column(Integer, ForeignKey('subject.subject_id'))
    marks = Column(Integer, nullable=False)
    student = relationship("Student", back_populates="student_details")
    subject = relationship("Subject", back_populates="student_details")


def create_table():
    Base.metadata.create_all(engine)
    tables = Base.metadata.tables.keys()
    print("Tables Created:")
    for table in list(tables):
        print(table)

def insert_sample_data(student_count, subject_count):
    Session = sessionmaker(bind=engine)
    session = Session()

    for i in range(student_count):
        student = Student(student_name=f"Student {i+1}")
        session.add(student)

    for i in range(subject_count):
        subject = Subject(subject_name=f"Subject {i+1}")
        session.add(subject)

    session.commit()

    students = session.query(Student).all()
    subjects = session.query(Subject).all()
    for student in students:
        for subject in subjects:
            marks = random.randint(0, 100)
            student_detail = StudentDetail(
                student_id=student.student_id,
                subject_id=subject.subject_id,
                marks=marks
            )
            session.add(student_detail)

    session.commit()
    print("Data inserted successfully.")

def print_table_data():
    tables = Base.metadata.tables.keys()
    for table in list(tables):
        print('Table Name:', table)
        df = pd.read_sql(f"SELECT * FROM {table}", con=engine)
        print(df)
        print()

def print_info_using_join():
    query = """
    SELECT s.student_name, sub.subject_name, sd.marks
    FROM student s
    JOIN student_detail sd ON s.student_id = sd.student_id
    JOIN subject sub ON sd.subject_id = sub.subject_id
    """
    df = pd.read_sql(query, con=engine)
    print('Students all details:')
    print(df)

def delete_table():
    Base.metadata.drop_all(engine)
    print("Tables Deleted.")

def delete_database():
    engine.dispose()
    print("Database connection closed.")

def main():
    create_table()
    insert_sample_data(5, 3)
    print_table_data()
    print_info_using_join()
    delete_table()
    delete_database()

if __name__ == '__main__':
    main()

Tables Created:
student
subject
student_detail
Data inserted successfully.
Table Name: student
   student_id student_name
0           1    Student 1
1           2    Student 2
2           3    Student 3
3           4    Student 4
4           5    Student 5

Table Name: subject
   subject_id subject_name
0           1    Subject 1
1           2    Subject 2
2           3    Subject 3

Table Name: student_detail
    id  student_id  subject_id  marks
0    1           1           1     85
1    2           1           2     78
2    3           1           3     68
3    4           2           1      7
4    5           2           2     26
5    6           2           3     87
6    7           3           1      4
7    8           3           2     73
8    9           3           3      6
9   10           4           1      8
10  11           4           2     45
11  12           4           3     84
12  13           5           1     77
13  14           5           2     73
14  15          

<ipython-input-20-789547450d31>:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
